<a href="https://www.kaggle.com/code/danielqinuk/gamstop-data-analysis?scriptVersionId=131029478" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/registered-businesses/business-licence-register-businesses.csv
/kaggle/input/operator-license/business-licence-register-licences.csv


In [2]:
#The dataset we need for our analysis is located from a url
#Add the dataset to kaggle using the url
#load the licence data

df_licence=pd.read_csv('/kaggle/input/operator-license/business-licence-register-licences.csv')
df_licence.head()


,Account Number,Licence Number,Status,Type,Activity,Start Date,End Date
0,102,000102-N-317976-007,Active,Non-Remote,Gaming Machine General Adult Gaming Centre,2014-03-12T00:00:00+00:00,NaN
1,103,000103-N-102731-016,Active,Non-Remote,Gaming Machine General Adult Gaming Centre,2009-01-01T00:00:00+00:00,NaN
2,103,000103-N-102731-016,Active,Non-Remote,Linked Gaming Machine Technical Supplier,2010-07-26T00:00:00+00:00,NaN
3,104,000104-N-100087-013,Active,Non-Remote,General Betting Limited,2009-01-01T00:00:00+00:00,NaN
4,109,000109-N-100074-008,Active,Non-Remote,General Betting Limited,2009-01-01T00:00:00+00:00,NaN


In [3]:
# Now we only want to keep the licences that are Remote type and 'Active'
df_licence=df_licence.loc[(df_licence['Status']=='Active')
& (df_licence['Type']=='Remote')]

df_licence.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1150 entries, 116 to 4732
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Account Number  1150 non-null   int64 
 1   Licence Number  1150 non-null   object
 2   Status          1150 non-null   object
 3   Type            1150 non-null   object
 4   Activity        1150 non-null   object
 5   Start Date      1150 non-null   object
 6   End Date        0 non-null      object
dtypes: int64(1), object(6)
memory usage: 71.9+ KB


In [4]:
#no data in Colunme'End Date' because we have filtered on 'Active' licence
#let's drop the 'End Date' column

df_licence.dropna(axis=1, inplace=True)
df_licence.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1150 entries, 116 to 4732
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Account Number  1150 non-null   int64 
 1   Licence Number  1150 non-null   object
 2   Status          1150 non-null   object
 3   Type            1150 non-null   object
 4   Activity        1150 non-null   object
 5   Start Date      1150 non-null   object
dtypes: int64(1), object(5)
memory usage: 62.9+ KB


In [5]:
#Only a few activites are pertinent to our analysis, so we will keep only those rows

df_licence=df_licence.loc[df_licence['Activity'].isin(['Casino','Bingo',
                                                       'External Lottery Manager',
                                                       'General Betting Standard - Virtual Event',
                                                       'General Betting Standard - Real Event',
                                                      'Pool Betting'])]
df_licence.head()

,Account Number,Licence Number,Status,Type,Activity,Start Date
116,400,000400-R-103839-026,Active,Remote,Casino,2018-06-11T00:00:00+00:00
117,400,000400-R-103839-026,Active,Remote,General Betting Standard - Real Event,2009-01-01T00:00:00+00:00
118,400,000400-R-103839-026,Active,Remote,General Betting Standard - Virtual Event,2020-05-12T00:00:00+00:00
119,400,000400-R-103839-026,Active,Remote,Pool Betting,2013-07-05T00:00:00+00:00
148,441,000441-R-103912-017,Active,Remote,Bingo,2009-01-01T00:00:00+00:00


In [6]:
# When we check the licence data, we can see some duplicates of licences. 
# This is because operator can register multiple activities under once licence
# Let's drop the duplicates of licences from the dataset and drop 'Activity' column as we have already finished filtering on it

df_list=df_licence.drop_duplicates(subset=['Licence Number'], keep='first')
df_list.drop(columns=['Activity'], inplace=True)
df_list.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 116 to 4707
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Account Number  247 non-null    int64 
 1   Licence Number  247 non-null    object
 2   Status          247 non-null    object
 3   Type            247 non-null    object
 4   Start Date      247 non-null    object
dtypes: int64(1), object(4)
memory usage: 11.6+ KB


/tmp/ipykernel_20/4262610364.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list.drop(columns=['Activity'], inplace=True)


In [7]:
# Our list has account number and licence number, but no business name.
# we will add these business names to the list by merging df_list and the business dataset
# Load the business dataset

df_business=pd.read_csv('/kaggle/input/registered-businesses/business-licence-register-businesses.csv')

# merge df_list and the business dataset
df_list=df_list.merge(df_business,how='inner',on='Account Number')

df_list.head()

,Account Number,Licence Number,Status,Type,Start Date,Licence Account Name
0,400,000400-R-103839-026,Active,Remote,2018-06-11T00:00:00+00:00,Fitzdares Limited
1,441,000441-R-103912-017,Active,Remote,2009-01-01T00:00:00+00:00,Palatial Leisure Limited
2,537,000537-R-103205-029,Active,Remote,2010-01-08T00:00:00+00:00,Genting Casinos UK Limited
3,584,000584-R-103711-013,Active,Remote,2009-01-01T00:00:00+00:00,CFP Lottery and Raffles Limited
4,614,000614-R-310112-011,Active,Remote,2011-02-08T00:00:00+00:00,Grosvenor Casinos Limited


In [8]:
# now we export the list to a CSV file

df_list.to_csv('operator_list.csv')